In [1]:
import pandas as pd
import os, glob, re

from owlready2 import *
import owlready2
print(owlready2.VERSION)


import importlib.util
import sys
spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

%load_ext autoreload
%autoreload 2


0.40


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
onto = get_ontology("../pbn_t3_5_v0.3.owl").load()
comments = u.checkComments(onto)

ID: 0 	 Author: Luc Jonveaux
ID: 1 	 License: CC BY-NC-SA
ID: 2 	 Language: English
ID: 3 	 Project: PROBONO
ID: 4 	 Task: T3.5
ID: 5 	 Description: Creation of a knowledge graph based on a litterature review, augmented by use of LLMs.
ID: 6 	 Changes from: 0.0
ID: 7 	 Changes: Creation of the knowledge graph
ID: 8 	 Changes from: 0.1
ID: 9 	 Changes: Adding relations, simplified ontology, closest items, and descriptions
ID: 10 	 Repository: https://github.com/mm80843/T3.5/
ID: 11 	 TODOs: Quite a number! Creating metaRisks, making it accessible..
ID: 12 	 Creation: 11/11/2023
ID: 13 	 Version: 0.3
ID: 14 	 VersionComment: New blueprints added in 0.3
ID: 15 	 Library: owlready2==0.45


In [3]:
for a in onto.Article.instances():
    LABELS = a.label
    LABELS = sorted(LABELS, key=len)
    a.label = [LABELS[-1].capitalize()]

In [4]:
u.cOnto(onto)


# Overview

*  _PBNThing_  --  19260 instances.
*  _BenefReturn_  --  1606 instances.
*  _Benef_  --  1409 instances.
*  _Article_  --  376 instances.
*  _Risk_  --  3418 instances.
*  _ISO_Scale_  --  5 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  2640 instances.
*  _Stakeholder_Type_  --  3 instances.
*  _Technology_  --  3915 instances.
*  _ISO_Impact_  --  24 instances.
*  _ISO_Purpose_  --  18 instances.
*  _Mitigation_  --  5295 instances.
*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _Blueprint_  --  74 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  40 instances.
*  _PBNCategory_  --  330 instances.
*  _BP_Theme_  --  10 instances.
*  _BP_Category_  --  3 instances.
*  _aBlueprint_  --  50 instances.


In [5]:
def cleanTitle(x):
    x = str(x).replace("("," ").replace(")"," ")
    x = re.sub(' +', ' ',  str(x)).strip().capitalize()
    return x

In [6]:
DOIs =[]
for a in onto.Article.instances():
    if a.label:
        a.label = [cleanTitle(a.label[0]) ]
    if a.has_ArticleDOI:
        A = a.label
        A = sorted(A, key=len)
        DOIs.append([a.has_ArticleDOI[0],A[0]])
dfDOI = pd.DataFrame(DOIs,columns=["doi","TITLE"])
#dfDOI.TITLE = dfDOI.TITLE.apply(lambda x: cleanTitle(x))

dfDOI["LEN"] = dfDOI.TITLE.apply(lambda x: len(x))
dfDOI.sort_values(by="LEN").head(10)

,doi,TITLE,LEN
26,10.1049/iet-smc.2020.0024,The smart city and covid‐19,27
342,10.1016/B978-0-12-818886-6.00016-2,Smart cities beyond covid-19,28
62,10.1177/2055207620920083,Covid-19: a new digital dawn?,29
351,10.1007/978-3-030-79515-3,The handbook of salutogenesis,29
98,10.1038/s41591-020-0824-5,Digital technology and covid-19,31
101,10.1016/S2213-2600(22)00351-4,Wearable technology and covid-19,32
223,10.1038/s41598-021-87692-z,Climate and the spread of covid-19,34
193,10.1007/s42399-020-00498-4,Covid-19 and its modes of transmission,38
363,10.15187/adr.2015.02.113.1.71,Biophilic design and bio-collaboration,38
38,10.1056/NEJMp2008017,Mental health and the covid-19 pandemic,39


In [7]:
dIDct = u.createDict(onto) 

In [8]:
dfR = pd.read_parquet("newrisks.parquet.gzip")
dfR = dfR.replace("","")
dfR = dfR.merge(dfDOI,on="doi",how="left").sort_values(by="TITLE")
dfR

/home/kelu/projets/T3.5/venv/lib/python3.10/site-packages/pandas/core/missing.py:106: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  new_mask[arr_mask] = arr[arr_mask] == x


,Description,Impact,Mitigation,Technologies,People,Owner,src,hash,author,title,doi,TITLE,LEN
3640,Potential transmission of contagious diseases ...,Increased risk of disease transmission among u...,"Encourage the use of individual devices, provi...","[Online access management systems, contactless...","[Library users, researchers, staff, students]","[Library management, facility administrators, ...",cheshmehzangi_10_2020,0bc460abde03230e831545f55f1f5258,"Cheshmehzangi, Ali",10 {Adaptive} {Measures} for {Public} {Places}...,10.1111/ciso.12282,10 adaptive measures for public places to face...,77
3639,Unverified Terms and Conditions,"Legal issues, copyright infringement",Review and adhere to verified terms and condit...,"[Legal review software, terms and conditions v...","[Users, researchers, institutions]","[Legal departments, research institutions]",cheshmehzangi_10_2020,6a3e092f19122ff2dc54ac3e5145a2d4,"Cheshmehzangi, Ali",10 {Adaptive} {Measures} for {Public} {Places}...,10.1111/ciso.12282,10 adaptive measures for public places to face...,77
3622,Temporary closure of access points may lead to...,Increased risk of non-compliance with safety m...,Implement clear communication of safety guidel...,"[Digital signage, public address systems]","[General public, visitors to public places]","[Public venue management, government authorities]",cheshmehzangi_10_2020,ebf8d4545fa00bceb645ccccc9196f2c,"Cheshmehzangi, Ali",10 {Adaptive} {Measures} for {Public} {Places}...,10.1111/ciso.12282,10 adaptive measures for public places to face...,77
3623,One-way mobility circulation may cause congest...,Increased risk of disease transmission due to ...,"Implement staggered entry and exit times, and ...","[Digital signage, time entry systems]","[General public, visitors to public places]","[Public venue management, government authorities]",cheshmehzangi_10_2020,ebf8d4545fa00bceb645ccccc9196f2c,"Cheshmehzangi, Ali",10 {Adaptive} {Measures} for {Public} {Places}...,10.1111/ciso.12282,10 adaptive measures for public places to face...,77
3624,Unchecked mobility in public places leading to...,Increased risk of disease transmission and out...,Implementing mobility control measures and che...,"[Surveillance technology, temperature scanning...","[General public, visitors, employees, shoppers]","[Government, public health authorities]",cheshmehzangi_10_2020,2dd8d2d8b1ed90de8cb255ff9e091891,"Cheshmehzangi, Ali",10 {Adaptive} {Measures} for {Public} {Places}...,10.1111/ciso.12282,10 adaptive measures for public places to face...,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13237,Increase in public reliance on waterhumandata ...,"Negative public perception, misinformation, an...",Implementing transparent communication strateg...,"[Data validation algorithms, real-time monitor...","[Citizens, policy makers, and water service pr...",[Public relations and communication profession...,poch_when_2020,6d247629ce6449ea956a3e74e38e6e59,"Poch, Manel and Garrido-Baserba, Manel and Cor...",When the fourth water and digital revolution e...,10.1016/j.scitotenv.2020.140980,When the fourth water and digital revolution e...,65
13238,Potential for digitalisation to compromise per...,Risk of privacy breaches and data exposure lea...,Implement robust data protection measures and ...,"[Data encryption, secure data storage systems,...","[Citizens, companies, water utility customers]","[Water utility companies, regulatory bodies]",poch_when_2020,c8737d7c3bb2626241a8bc17b8b680c5,"Poch, Manel and Garrido-Baserba, Manel and Cor...",When the fourth water and digital revolution e...,10.1016/j.scitotenv.2020.140980,When the fourth water and digital revolution e...,65
13254,Digital technology extracting intimate health ...,"Privacy concerns, potential misuse of personal...","Implement stringent data privacy regulations, ...","[Data encryption, secure data storage and tran...","[General public, data protection authorities, ...","[Regulatory bodies, technology companies, and ..."

In [9]:
with onto:
    class has_bibRef(onto.Article >> str):
        label = ["Bibtex reference"]
        pass

In [10]:

for ix, row in dfR.iterrows():
    if ix < 20000:
        try:
            newRisk =  u.addItem("Risk", row["Description"], dIDct, onto )
            newRisk.has_RiskName.append(row["Impact"])

            src= u.addItem("Article",row["TITLE"], dIDct, onto )
            src.has_bibRef.append(row["src"])
            src.has_bibRef = list(set(src.has_bibRef))
            newRisk.has_RiskSource.append(src)

            mitig = u.addItem("Mitigation",row["Mitigation"], dIDct, onto )
            newRisk.has_RiskMitigation.append(mitig)
            mitig.has_MitigationSource.append(src)
            
            for tech in row["Technologies"]:
                mytech = u.addItem("Technology",tech, dIDct, onto ) 
                newRisk.has_RiskTechnology.append(mytech)
                mytech.has_TechSource.append(src)

            for stakeholder in row["People"]:
                newstakeholder = u.addItem("Stakeholder",stakeholder, dIDct, onto )  
                newRisk.has_RiskSubject.append(newstakeholder)
                                
            for stakeholder in row["Owner"]:
                newstakeholder = u.addItem("Stakeholder",stakeholder, dIDct, onto )   
                newRisk.has_RiskOwner.append(newstakeholder)
        except:
            pass


In [11]:
# CHecking for empty articles
DOIs =[]
for a in onto.Article.instances():
    DOIs.append([a.has_ArticleDOI,a.has_ArticleTitle,a.label[0]])
dfDOIs = pd.DataFrame(DOIs,columns=["doi","TITLE","titre"]).sort_values(by="titre")
dfDOIs

,doi,TITLE,titre
167,[10.1111/ciso.12282],[10 Adaptive Measures for Public Places to fac...,10 adaptive measures for public places to face...
188,[10.1128/mSystems.00245-20],[2019 Novel Coronavirus (COVID-19) Pandemic: B...,2019 novel coronavirus covid-19 pandemic: buil...
318,[10.1016/j.ypmed.2021.106425],[A call to action: Improving urban green space...,A call to action: improving urban green spaces...
212,[10.1088/1757-899x/1055/1/012083],[A Comprehensive Review of Applications of Int...,A comprehensive review of applications of inte...
280,[10.1109/ACCESS.2020.2992341],[A Comprehensive Review of the COVID-19 Pandem...,A comprehensive review of the covid-19 pandemi...
...,...,...,...
307,[10.1080/0960085X.2020.1770632],[What drives unverified information sharing an...,What drives unverified information sharing and...
163,[10.1787/65cfc31c-en],[What has been the impact of the COVID-19 pand...,What has been the impact of the covid-19 pande...
227,[10.1088/1748-9326/abb851],[What our response to the COVID-19 pandemic te...,What our response to the covid-19 pandemic tel...
4,[10.1016/j.scitotenv.2020.140980],[When the fourth water and digital revolution ...,When the fourth water and digital revolution e...


In [12]:
u.cOnto(onto)


# Overview

*  _PBNThing_  --  86872 instances.
*  _BenefReturn_  --  1606 instances.
*  _Benef_  --  1409 instances.
*  _Article_  --  377 instances.
*  _Risk_  --  20899 instances.
*  _ISO_Scale_  --  5 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  16002 instances.
*  _Stakeholder_Type_  --  3 instances.
*  _Technology_  --  23093 instances.
*  _ISO_Impact_  --  24 instances.
*  _ISO_Purpose_  --  18 instances.
*  _Mitigation_  --  22885 instances.
*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _Blueprint_  --  74 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  40 instances.
*  _PBNCategory_  --  330 instances.
*  _BP_Theme_  --  10 instances.
*  _BP_Category_  --  3 instances.
*  _aBlueprint_  --  50 instances.


In [13]:
onto.save("./WIP_377.owl")

In [14]:
print("Done")

Done
